# Import packages

In [6]:
import requests
import json
import pandas as pd

In [340]:
url = 'https://www.brilliantearth.com/loose-diamonds/list/?shapes=Round&min_carat=0.60&max_carat=0.60&page=1'

In [341]:
response = requests.get(url).json()

In [342]:
print(f'total_count: {response["total_count"]}')
print(f'page: {response["page"]}')
print(f'num_records: {len(response["diamonds"])}')
print(f'path: {response["path"]}')

total_count: 1012
page: 1
num_records: 20
path: ?shapes=Round&min_carat=0.60&max_carat=0.60&page=1


In [178]:
print(f"***** Retrieving {response['total_count']:,} carat diamonds *****")

***** Retrieving 16,423 carat diamonds *****


In [ ]:
print(f"index_id: {response['diamonds'][1]['index_id']}")
print(f"id: {response['diamonds'][1]['id']}")

In [23]:
test_url = 'https://www.brilliantearth.com/loose-diamonds/list/'
test_path = '?shapes=Round&min_carat={self.min_carat}&max_carat={self.max_carat}&page={self.page_num}'

test_full_path = test_url + test_path
test_full_path

'https://www.brilliantearth.com/loose-diamonds/list/?shapes=Round&min_carat={self.min_carat}&max_carat={self.max_carat}&page={self.page_num}'

In [343]:
class Diamonds:
    def __init__(self, page_num=1, min_carat=0.60, max_carat=0.60):

        self.page_num = page_num
        self.min_carat = min_carat
        self.max_carat = max_carat
        
        self.url = 'https://www.brilliantearth.com/loose-diamonds/list/'
        self.num_records = 0
        
        self.diamond_id = []
        self.carat = []
        self.color = []
        self.clarity = []
        self.cut = []
        self.price = []

    
    def get_diamonds(self):
        
        path = f'?shapes=Round&min_carat={self.min_carat}&max_carat={self.min_carat}&page={self.page_num}'
        full_path = self.url + path
        
        response = requests.get(full_path).json()

        for diamond in response['diamonds']:
            self.diamond_id.append(diamond['id'])
            self.carat.append(diamond['carat'])
            self.color.append(diamond['color'])
            self.clarity.append(diamond['clarity'])
            self.cut.append(diamond['cut'])
            self.price.append(diamond['price'])
            self.num_records += 1
            
#         while self.page_num < 14:
#             self.page_num += 1
#             self.get_diamonds()
            

        if len(response['diamonds']) == 20:
            print(f'Page {self.page_num} done.')
            self.page_num += 1
            self.get_diamonds()
            
        else:
            print(f'***** Retrived {self.num_records:,} diamonds of {self.min_carat:.2f} carat weight *****')

#         if self.min_carat < self.max_carat:
#             self.min_carat += 0.01
#             self.page_num = 1
#             self.get_diamonds()

#             if len(self.upc) < data['total_count']:
#                 self.page_num += 1
#                 self.get_diamonds()
            
#         if self.max_carat < self.upper_limit: 
#             self.min_carat += 0.1
#             self.max_carat += 0.1
#             self.page_num = 1
#             self.get_diamonds()

    def create_frame(self):
        
        data = {
            'diamond_id': self.diamond_id,
            'carat': self.carat,
            'color': self.color,
            'clarity':self.clarity,
            'cut': self.cut,
            'price': self.price
        }
        
        return pd.DataFrame(data)
    

In [344]:
bling = Diamonds()

In [345]:
%%time
bling.get_diamonds()

Page 1 done.
Page 2 done.
Page 3 done.
Page 4 done.
Page 5 done.
Page 6 done.
Page 7 done.
Page 8 done.
Page 9 done.
Page 10 done.
Page 11 done.
Page 12 done.
Page 13 done.
Page 14 done.
Page 15 done.
Page 16 done.
Page 17 done.
Page 18 done.
Page 19 done.
Page 20 done.
Page 21 done.
Page 22 done.
Page 23 done.
Page 24 done.
Page 25 done.
Page 26 done.
Page 27 done.
Page 28 done.
Page 29 done.
Page 30 done.
Page 31 done.
Page 32 done.
Page 33 done.
Page 34 done.
Page 35 done.
Page 36 done.
Page 37 done.
Page 38 done.
Page 39 done.
Page 40 done.
Page 41 done.
Page 42 done.
Page 43 done.
Page 44 done.
Page 45 done.
Page 46 done.
Page 47 done.
Page 48 done.
Page 49 done.
Page 50 done.
***** Retrived 1,012 diamonds of 0.60 carat weight *****
CPU times: user 1.22 s, sys: 114 ms, total: 1.34 s
Wall time: 28.9 s


In [346]:
len(bling.diamond_id)

1012

In [347]:
diamonds0 = bling.create_frame()

In [348]:
diamonds0.head()

,diamond_id,carat,color,clarity,cut,price
0,10166957,0.6,G,SI2,Very Good,1150
1,10231299,0.6,F,SI2,Good,1170
2,9602673,0.6,J,SI1,Very Good,1190
3,9242137,0.6,I,SI1,Good,1220
4,10221236,0.6,H,SI2,Good,1260


In [349]:
len(diamonds0['diamond_id'].unique())

1012

In [350]:
df5 = pd.concat(
    [df4, diamonds0],
    ignore_index=True
)

In [352]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37705 entries, 0 to 37704
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   diamond_id  37705 non-null  int64  
 1   carat       37705 non-null  float64
 2   color       37705 non-null  object 
 3   clarity     37705 non-null  object 
 4   cut         37705 non-null  object 
 5   price       37705 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 1.7+ MB


In [215]:
# diamonds.duplicated(subset='upc')
diamonds[diamonds.duplicated(['diamond_id'], keep=False)]

,diamond_id,carat,color,clarity,cut,price
259,9563097,0.25,F,VS1,Super Ideal,620
261,9563097,0.25,F,VS1,Super Ideal,620
4270,10160929,0.30,D,VS1,Very Good,780
4271,10160929,0.30,D,VS1,Very Good,780
17914,10205003,0.37,D,VS1,Ideal,1110
...,...,...,...,...,...,...
58354,10238504,0.43,J,VVS1,Super Ideal,990
58355,9588287,0.43,E,SI1,Very Good,1000
58356,9966456,0.43,D,SI2,Super Ideal,1010
58357,10186043,0.43,J,VVS1,Super Ideal,1010


In [315]:
df2 = pd.concat(
    [diamonds0, diamonds1, diamonds2, diamonds3, diamonds4, diamonds5, diamonds6,  diamonds7, diamonds8, diamonds9],
    ignore_index=True
)

In [316]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10458 entries, 0 to 10457
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   diamond_id  10458 non-null  int64  
 1   carat       10458 non-null  float64
 2   color       10458 non-null  object 
 3   clarity     10458 non-null  object 
 4   cut         10458 non-null  object 
 5   price       10458 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 490.3+ KB


In [199]:
df.tail()

,diamond_id,carat,color,clarity,cut,price
25984,10191979,0.49,F,VS1,Very Good,1940
25985,10223274,0.49,E,VVS1,Super Ideal,2040
25986,10099446,0.49,D,VS2,Super Ideal,2170
25987,9598921,0.49,D,VVS1,Super Ideal,2230
25988,9951008,0.49,D,IF,Super Ideal,2780


In [321]:
df3 = pd.concat(
    [df, df2],
    ignore_index=True
)

In [322]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36447 entries, 0 to 36446
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   diamond_id  36447 non-null  int64  
 1   carat       36447 non-null  float64
 2   color       36447 non-null  object 
 3   clarity     36447 non-null  object 
 4   cut         36447 non-null  object 
 5   price       36447 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 1.7+ MB


In [353]:
df5.to_csv('diamonds.csv', index=False)

In [324]:
df3.tail()

,diamond_id,carat,color,clarity,cut,price
36442,10236892,0.58,E,VVS1,Super Ideal,3850
36443,9922838,0.58,D,IF,Super Ideal,3980
36444,9965270,0.58,D,IF,Super Ideal,4310
36445,10209495,0.58,D,FL,Super Ideal,4410
36446,10020378,0.58,D,FL,Super Ideal,5030
